In [1]:
#tkinter
import tkinter as tk
import tkinter.ttk as ttk
import threading
from PIL import ImageTk, Image
from functools import partial

#selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from urllib.parse import unquote

#other
import time
import json
import os
import import_ipynb

#show image
from IPython.core.debugger import set_trace
from IPython.display import Image as iImage
from IPython.core.display import HTML
from IPython.core.debugger import set_trace
from IPython.display import clear_output as clear
from urllib.request import urlopen
from io import BytesIO

class ComicFuz:
    
    def __init__(self,window,driver): 
#         self.wdsz = wdsz
        self.window = window
        self.driver = driver
        self.window.configure(background = 'black')
        
        self.login_frame = tk.Frame(self.window)
        self.bg = 'black'
        self.fg = 'orange'
        self.ft = 'Helvetica 12 bold'
        
        self.login_window()
        
        print("noooooo")
        
        #scrollable things
        self.main_canvas = tk.Canvas(self.window)
        self.main_canvas.pack(expand = True,side = 'left',fill = 'both')
        
        self.main_scrollbar = tk.Scrollbar(self.window,orient = 'vertical',command = self.main_canvas.yview)
        #self.main_scrollbar.pack(side = 'left',fill = 'y')
        
        self.main_canvas.configure(yscrollcommand = self.main_scrollbar.set)
        self.main_canvas.bind('<Configure>',self.on_configure)
        self.main_canvas.bind_all('<MouseWheel>', self._on_mousewheel)
        
        #need to add width,height information for centering
        self.list_frame = tk.Frame(self.main_canvas,width = 500,height = 300)
        self.list_frame.configure(bg = 'black')
        self.list_frame.pack(expand = True,side = 'left',fill = 'both')
        self.main_canvas.configure(bg = 'black')
        self.main_canvas.create_window((0,0),window = self.list_frame,anchor = 'nw')
        
        #manga cover list (image)
        self.readable_manga_cover = []
        #manga title list
        self.readable_manga_title = []
        #manga link list
        self.readable_link_list = []
        
    def check_thread_alive(self,thread):
        if thread.is_alive():
            self.window.after(20,self.check_thread_alive,thread)
        else :
            pass
        
    def start_btn_login_check(self,usr,pwd,fr,wd):
        
        th = threading.Thread(target = self.web_login,args = (usr.get(),pwd.get(),fr,wd,))
        th.setDaemon(True)
        th.start()
        self.window.after(20,self.check_thread_alive,th)
        #th.join()
        
#     def login_check(self,usr,pwd,fr,wd):
#         #print('in')
        
#         global var_suc
        
#         #print("usr : " + usr.get())
#         #print("pwd : " + pwd.get())
        
#         chrome_options = Options()
#         chrome_options.add_argument('headless')
        
#         web_login = ComicFuzLogin(webdriver.Chrome(options = chrome_options))
        
#         web_login.get_login_cookie(usr.get(),pwd.get())
#         #login_success = web_login.get_login_cookie(usr.get(),pwd.get())
        
#         login_success = web_login.login()
    
#         #adjust to check
#         if login_success:
#             var_suc.set('Login Successfully.')
#             wd.update()
#             time.sleep(2)
#             fr.destroy()
#         else :
#             var_suc.set('Fail...')
            
#         #print('back')
        
    def login_window(self):
        
#         global window
#         window.title('Login')
#         window.geometry(self.wdsz)
#         window.resizable(0,0)

        self.login_frame.pack(expand = True,fill = 'both')
        self.login_frame.configure(background = self.bg)

        #control size of image
        p = 0.5
        #global font_param
        #font_param = 'Helvetica 12 bold'
        
        #image path
        pre_img = Image.open('Cover.png')
        
        #get old size value
        pre_w , pre_h = pre_img.size[0] , pre_img.size[1]
        
        #resize it
        pre_img = pre_img.resize((int(p * pre_w),int(p * pre_h)),Image.ANTIALIAS)
        img = ImageTk.PhotoImage(pre_img)
        
        print(type(img))
        print(img.width(),img.height())
        
        front_img = tk.Label(self.login_frame,image = img)
        front_img.photo = img    #important line to show image
        front_img.pack()
#         front_img = tk.Canvas(self.main_frame)
#         front_img.create_image(0,0,image = img)
#         front_img.pack(side = 'top')

        #prompt text
        login_title = tk.Label(self.login_frame,text = "It's a program that helps you download your manga.",
                            bg = self.bg,fg = self.fg,height = 2,font = self.ft)
        login_title.pack()

        #entry frame
        btm_frame = tk.Frame(self.login_frame)
        btm_frame.pack(side = 'top',fill = 'y')
        btm_frame.configure(background = self.bg)

        #setting a grid
        usr_text = tk.Label(btm_frame,text = 'mail-address : ',bg = 'dimgrey',fg = self.fg,font = self.ft)
        pwd_text = tk.Label(btm_frame,text = 'password : ',bg = 'dimgrey',fg = self.fg,font = self.ft)


        usr_text.grid(row = 0,sticky = 'nsew')
        pwd_text.grid(row = 1,sticky = 'nsew')

        #btm_frame.grid_columnconfigure(2,weight = 1)
        #btm_frame.grid_rowconfigure(2,weight = 1)

        usr_entry = tk.Entry(btm_frame,font = self.ft)
        pwd_entry = tk.Entry(btm_frame,show = '*',font = self.ft)

        usr_entry.grid(row = 0,column = 1)
        pwd_entry.grid(row = 1,column = 1)

        login_btn = tk.Button(self.login_frame,text = 'submit',bg = self.bg,fg = self.fg,
                           font = self.ft,command = partial(self.start_btn_login_check,usr_entry,pwd_entry,self.login_frame,self.window))
        login_btn.pack(pady = 10)
        
        global var_suc
        var_suc = tk.StringVar()
        
        login_suc_text = tk.Label(self.login_frame,textvariable = var_suc,bg = self.bg,fg = self.fg,height = 2,font = self.ft)
        login_suc_text.pack(expand = True,fill = 'x')
    
    
    #list comic
    #webdriver
    
    def use_json_login(self):
        login_json_path =  'comicFuz.json'

        if os.path.isfile(login_json_path):
            print("Already login before")
            print("Skip login part\n")
        else :
            print('You haven\'t login yet')
            print('Please login at least once')
            self.get_login_cookie()
        
    def web_login(self,account,password,fr,wd):
        
        progress_window = tk.Toplevel(wd)
        progress_bar = ttk.Progressbar(progress_window,orient='horizontal',length = 100,mode = 'determinate')
        progress_bar.pack()
        
        url = "https://comic-fuz.com/accounts/my_page"
        #driver = webdriver.Chrome()
        self.driver.get(url)
        
        progress_bar['value'] = 30
        wd.update_idletasks()
        
        #user input
        
        #login_success = False
        
        #while login_success == False:
            
        login_wait = WebDriverWait(self.driver,10).until(
            EC.presence_of_element_located((By.XPATH,'/html/body/main/section/div/div/div/section[1]/div/form/div/input'))
        )

        progress_bar['value'] = 50
        wd.update_idletasks()
        
        user_account_block = self.driver.find_element_by_id('account_email')
        user_password_block = self.driver.find_element_by_id('account_password')
        
        print('in login page')

#             user_account = input('Enter your ComicFuz account (email address) : ')
#             user_password = input('Enter your ComicFuz password : ')

        user_account = account
        user_password = password

#         user_account_block.clear()
#         user_password_block.clear()

        user_account_block.send_keys(user_account)
        user_password_block.send_keys(user_password)
        
        progress_bar['value'] = 75
        wd.update_idletasks()

        self.driver.find_element_by_xpath("/html/body[@class='fuz_accounts_sessions']/main/section[@id='login']/div[@class='inner']/div[@id='main']/div[@class='column_primary']/section[@id='member']/div[@id='sign_in_box']/form[@id='new_account']/div[@class='box_button']/input[@class='btn btn-default button button_decision']").send_keys(Keys.ENTER)
        #time.sleep(2)
        
        global var_suc
        
        try:
            login_finish = WebDriverWait(self.driver,6).until(
                EC.presence_of_element_located((By.XPATH,'/html/body/main/div/div/div/div[1]/div/div/ul/ul[2]/li[1]/a'))
            )
            #some login logic here
            #self.driver.find_element_by_xpath('/html/body/main/div/div/div/div[1]/div/div/ul/ul[2]/li[1]/a')
            print('Login successfully!')
            #login_success = True
            
            time.sleep(2)
            
            progress_bar['value'] = 100
            wd.update_idletasks()

            cookie = self.driver.get_cookies()

            json_cookies = json.dumps(cookie)

            with open('comicFuz.json','w') as f:
                f.write(json_cookies)
            
            progress_window.destroy()
            
            var_suc.set('Login Successfully.')
            wd.update_idletasks()
            time.sleep(2)
            fr.destroy()
            
            self.start_list_of_readable_manga()
            
            return True
            
        except:
            print('Login failed...')
            print('Please try again!')
            
            var_suc.set('Fail...')
            
            return False
        
        
    #scrollable things
    def on_configure(self,event):
        self.main_canvas.configure(scrollregion = self.main_canvas.bbox("all"))
        
    def _on_mousewheel(self, event):
        self.main_canvas.yview_scroll(int(-1*(event.delta/50)), "units")
        
    def cover_link_to_image(self,link):
        
        p = 0.2
        
        for i in range(len(link)):
            image_url = urlopen(link[i])
            url_data = image_url.read()
            image_url.close()
            link[i] = Image.open(BytesIO(url_data))

            pre_w , pre_h = link[i].size[0] , link[i].size[1]
            link[i] = link[i].resize((int(p * pre_w),int(p * pre_h)),Image.ANTIALIAS)
            
            self.readable_manga_cover.append(ImageTk.PhotoImage(link[i]))
            
            print(i,'th cover complete')
            
        print('all complete')
    
    def list_all_in_frame(self):
#         #scrollable things
#         self.main_canvas = tk.Canvas(self.window)
#         self.main_canvas.pack(expand = True,side = 'left',fill = 'both')
        
#         self.main_scrollbar = tk.Scrollbar(self.window,orient = 'vertical',command = self.main_canvas.yview)
#         #self.main_scrollbar.pack(side = 'left',fill = 'y')
        
#         self.main_canvas.configure(yscrollcommand = self.main_scrollbar.set)
#         self.main_canvas.bind('<Configure>',self.on_configure)
#         self.main_canvas.bind_all('<MouseWheel>', self._on_mousewheel)
        
#         #need to add width,height information for centering
#         self.main_frame = tk.Frame(self.main_canvas,width = 500,height = 300)
#         self.main_frame.configure(bg = 'black')
#         self.main_frame.pack(expand = True,side = 'left',fill = 'both')
#         self.main_canvas.configure(bg = 'black')
#         self.main_canvas.create_window((0,0),window = self.main_frame,anchor = 'nw')
        
        #show image
        
        #shukan times : 10 ,(0 ~ 9) for self
        
        shukan_times_title = tk.Label(self.list_frame,text = "週刊漫画ＴＩＭＥＳ",bg = self.bg,fg = self.fg,font = self.ft)
        shukan_times_title.grid(row = 0,column = 0,columnspan = 2,sticky = 'news')
        
        shukan_times = []
        shukan_times_subtitle = []
        
        row_cnt = 1
        
        for i in range(10):
            if i % 2 == 0 and i != 0:
                row_cnt += 2
                
            if "週刊漫画ＴＩＭＥＳ　" in self.readable_manga_title[i]:
                self.readable_manga_title[i] = self.readable_manga_title[i].replace("週刊漫画ＴＩＭＥＳ　","")

            shukan_times.append(tk.Label(self.list_frame,image = self.readable_manga_cover[i],bg = self.bg))
            shukan_times_subtitle.append(tk.Button(self.list_frame,text = self.readable_manga_title[i],bg = self.bg,fg = self.fg,font = "Helvetica 8 bold",
                                                   command = partial(self.start_download_comic,i)))
            
            shukan_times[i].photo = self.readable_manga_cover[i]
            
            shukan_times[i].grid(row = row_cnt,column = i % 2,sticky = 'news')
            shukan_times_subtitle[i].grid(row = row_cnt + 1,column = i % 2,sticky = 'news')
        
        
        #manga_time_kirara : 3 ,(10 ~ 12) for self
        
        manga_time_kirara_title = tk.Label(self.list_frame,text = "まんがタイムきらら",bg = self.bg,fg = self.fg,font = self.ft)
        manga_time_kirara_title.grid(row = row_cnt + 2,column = 0,columnspan = 2,sticky = 'news')
        
        row_cnt += 3
        
        manga_time_kirara = []
        manga_time_kirara_subtitle = []
        
        for i in range(3):
            if i % 2 == 0 and i != 0:
                row_cnt += 2
            
            j = i + 10
            
            if "まんがタイムきらら　" in self.readable_manga_title[j]:
                self.readable_manga_title[j] = self.readable_manga_title[j].replace("まんがタイムきらら　","")
            
            manga_time_kirara.append(tk.Label(self.list_frame,image = self.readable_manga_cover[j],bg = self.bg))
            manga_time_kirara_subtitle.append(tk.Button(self.list_frame,text = self.readable_manga_title[j],bg = self.bg,fg = self.fg,font = "Helvetic 8 bold",
                                                        command = partial(self.start_download_comic,j)))
            
            #adjust idx
            manga_time_kirara[i].photo = self.readable_manga_cover[j]
            
            manga_time_kirara[i].grid(row = row_cnt,column = i % 2,sticky = 'news')
            manga_time_kirara_subtitle[i].grid(row = row_cnt + 1,column = i % 2,sticky = 'news')
            
            
        #manga_time_kirara_max : 3 ,(13 ~ 15) for self
        
        manga_time_kirara_max_title = tk.Label(self.list_frame,text = "まんがタイムきららＭＡＸ",bg = self.bg,fg = self.fg,font = self.ft)
        manga_time_kirara_max_title.grid(row = row_cnt + 2,column = 0,columnspan = 2,sticky = 'news')
        
        row_cnt += 3
        
        manga_time_kirara_max = []
        manga_time_kirara_max_subtitle = []
        
        for i in range(3):
            if i % 2 == 0 and i != 0:
                row_cnt += 2
            
            j = i + 13
            
            if "まんがタイムきららＭＡＸ　" in self.readable_manga_title[j]:
                self.readable_manga_title[j] = self.readable_manga_title[j].replace("まんがタイムきららＭＡＸ　","")
            
            manga_time_kirara_max.append(tk.Label(self.list_frame,image = self.readable_manga_cover[j],bg = self.bg))
            manga_time_kirara_max_subtitle.append(tk.Button(self.list_frame,text = self.readable_manga_title[j],bg = self.bg,fg = self.fg,font = "Helvetic 8 bold",
                                                            command = partial(self.start_download_comic,j)))
            
            #adjust idx
            manga_time_kirara_max[i].photo = self.readable_manga_cover[j]
            
            manga_time_kirara_max[i].grid(row = row_cnt,column = i % 2,sticky = 'news')
            manga_time_kirara_max_subtitle[i].grid(row = row_cnt + 1,column = i % 2,sticky = 'news')
            
        
        #manga_time_kirara_carat : 3 ,(16 ~ 18) for self
        
        manga_time_kirara_carat_title = tk.Label(self.list_frame,text = "まんがタイムきららキャラット",bg = self.bg,fg = self.fg,font = self.ft)
        manga_time_kirara_carat_title.grid(row = row_cnt + 2,column = 0,columnspan = 2,sticky = 'news')
        
        row_cnt += 3
        
        manga_time_kirara_carat = []
        manga_time_kirara_carat_subtitle = []
        
        for i in range(3):
            if i % 2 == 0 and i != 0:
                row_cnt += 2
            
            j = i + 16
            
            if "まんがタイムきららキャラット　" in self.readable_manga_title[j]:
                self.readable_manga_title[j] = self.readable_manga_title[j].replace("まんがタイムきららキャラット　","")
            
            manga_time_kirara_carat.append(tk.Label(self.list_frame,image = self.readable_manga_cover[j],bg = self.bg))
            manga_time_kirara_carat_subtitle.append(tk.Button(self.list_frame,text = self.readable_manga_title[j],bg = self.bg,fg = self.fg,font = "Helvetic 8 bold",
                                                              command = partial(self.start_download_comic,j)))
            
            #adjust idx
            manga_time_kirara_carat[i].photo = self.readable_manga_cover[j]
            
            manga_time_kirara_carat[i].grid(row = row_cnt,column = i % 2,sticky = 'news')
            manga_time_kirara_carat_subtitle[i].grid(row = row_cnt + 1,column = i % 2,sticky = 'news')
            
        
        #manga_time_kirara_forward : 3 ,(19 ~ 21) for self
        
        manga_time_kirara_forward_title = tk.Label(self.list_frame,text = "まんがタイムきららフォワード",bg = self.bg,fg = self.fg,font = self.ft)
        manga_time_kirara_forward_title.grid(row = row_cnt + 2,column = 0,columnspan = 2,sticky = 'news')
        
        row_cnt += 3
        
        manga_time_kirara_forward = []
        manga_time_kirara_forward_subtitle = []
        
        for i in range(3):
            if i % 2 == 0 and i != 0:
                row_cnt += 2
            
            j = i + 19
            
            if "まんがタイムきららフォワード　" in self.readable_manga_title[j]:
                self.readable_manga_title[j] = self.readable_manga_title[j].replace("まんがタイムきららフォワード　","")
            
            manga_time_kirara_forward.append(tk.Label(self.list_frame,image = self.readable_manga_cover[j],bg = self.bg))
            manga_time_kirara_forward_subtitle.append(tk.Button(self.list_frame,text = self.readable_manga_title[j],bg = self.bg,fg = self.fg,font = "Helvetic 8 bold",
                                                                command = partial(self.start_download_comic,j)))
            
            #adjust idx
            manga_time_kirara_forward[i].photo = self.readable_manga_cover[j]
            
            manga_time_kirara_forward[i].grid(row = row_cnt,column = i % 2,sticky = 'news')
            manga_time_kirara_forward_subtitle[i].grid(row = row_cnt + 1,column = i % 2,sticky = 'news')
            
#         self.main_canvas.pack(expand = True,side = 'left',fill = 'both')
        self.main_scrollbar.pack(side = 'left',fill = 'y')
#         self.main_frame.pack(expand = True,side = 'left',fill = 'both')

        print('over')
    
    
    # webdriver
    
    def check_thread_alive(self,thread):
        if thread.is_alive():
            self.window.after(20,self.check_thread_alive,thread)
        else :
            pass

    #download comic part
    
    def make_dir(self,dir_name,num,total_page,text): #add manga type param
        title = ['週刊漫画ＴＩＭＥＳ','まんがタイムきらら','まんがタイムきららＭＡＸ'
                 ,'まんがタイムきららキャラット','まんがタイムきららフォワード']

        filename = "D:\Picture\ComicFuz\\"+ title[num] + "\\" + dir_name
        
        page = 1

        if os.path.isdir(filename):
            file_num = [f for f in os.listdir(filename) if os.path.isfile(filename + "\\" + f)]

        if not os.path.isdir(filename):
            print("create a folder " + filename + " successfully\n")
            os.makedirs(filename)

        elif len(file_num) == total_page:
            print('Already downloaded.')
#             download = input("Already downloaded.")
#             if download != 'Y' and download != 'y':
#                 exit()

        else :
            print('You only download',str(len(file_num)) + '/' + str(total_page))
            page = file_num
            download = input('Do you need to download again? [Y/n]')
            if download != 'Y' and download != 'y':
                exit()

        return filename , page

    
    #decide which class this manga using
    
    def first_page_class(self):
        #sometimes first page with viewport0
        #sometimes with viewport1
        try:
            element = self.driver.find_element_by_xpath('//*[contains(@class,"currentScreen")]')
            name = element.get_attribute("id")
            if name[-1] == '0':
                return 0
            else :
                return 1
        except:
            print("Something went wrong...")
            return 0
    
    #wait for comic page loaded
    
    def comic_page_loaded(self):
        try:
            print("Waiting for page loaded.\n")
            canvas_xpath = '//*[@id="viewport1"]/canvas'
            comic_page = WebDriverWait(driver,15).until(
                EC.presence_of_element_located((By.XPATH,canvas_xpath))
                #lambda driver : driver.find_element_by_xpath(element_xpath)
            )

        finally:
            print("Web is loaded")
    
    #change the default(show two page in one screen) to show one page only
    
    def two_page_to_one(self):
        #change two page one screen to one pg one scr

        bar_path = '//*[contains(@class,"hidden twoButton")]'
        wait_bar_dis = WebDriverWait(self.driver,10).until(
            EC.presence_of_element_located((By.XPATH,bar_path))
        )

        ActionChains(driver).click().perform()

        self.driver.find_element_by_xpath('/html/body/div[2]/div/div[4]/div[2]/div[3]/button').send_keys(Keys.ENTER)

        one_page_btn = self.driver.find_element_by_xpath('/html/body/div[7]/div[3]/div[3]/span/label[1]')
        one_page_hover = ActionChains(driver).move_to_element(one_page_btn)
        one_page_hover.click().perform()

        self.driver.find_element_by_xpath('/html/body/div[2]/div/div[4]/div[2]/div[3]/button').send_keys(Keys.ENTER)
        
        print("\nTwo page to one page\n")
        
    #move to first page   
    
    def to_comic_first_page(self):
        
        page_now_ele = self.driver.find_element_by_xpath('/html/body/div[2]/div/div[6]/div/div[2]/div/div[1]/div/div[2]/div')
        page_now_str = ""
        total_page_str = ""
        chg = 0

        for i in page_now_ele.text:
            if i == '/':
                chg = 1
                continue
            if chg == 0:    
                page_now_str += i
            else :
                total_page_str += i
        
        print("page_now_ele.text :",page_now_ele.text)
        print("page_now_str",page_now_str,type(page_now_str))
        
        page_now = int(page_now_str)
        total_page = int(total_page_str)

        #print((page_num - 1))

        for i in range((page_now - 1)):
            time.sleep(0.2)
            page_to_first = ActionChains(driver).send_keys(Keys.RIGHT).perform()

        print("Already to the first page.\n")
        
        return page_now_ele , total_page
    
    #wait for comic progress bar disappear
    
    def comic_bar_disappear(self):
        try:
            print("Waiting for bar to disappear.\n")
#             element_xpath = '/html/body/div[2]/div/div[4]'
#             element = WebDriverWait(self.driver,15).until(
#                 EC.presence_of_element_located((By.XPATH, "//*[@id='menu']"))
#             )
            
#             bar_xpath = '/html/body/div[2]/div/div[6]'
#             bar = WebDriverWait(self.driver,15).until_not()

            menu_bar = WebDriverWait(driver,15).until_not(
                EC.presence_of_element_located((By.XPATH,'//div[contains(@class,"hidden twoButton show") and contains(@id,"menu")]'))
            )

            cotent_bar = WebDriverWait(driver,15).until_not(
                EC.presence_of_element_located((By.XPATH,'//div[contains(@class,"hidden show") and contains(@id,"pageSlider")]'))
            )

        finally:
            print("Already disappear.")
            
    #create a new dir for manga        
            
    def new_dir_name(self,url):
        window_width = self.driver.execute_script("return document.documentElement.scrollWidth")
        window_height = self.driver.execute_script("return document.documentElement.scrollHeight")
        print("Window size :",window_width,window_height,"\n")

        self.driver.set_window_size(window_width,window_height + 550)
        time.sleep(1)

        st = 0
        title = ""

        for i in range(4,len(url)):
            if (url[i] == '%') or st:
                if url[i] == '&':
                    break
                st = 1
                title += url[i]
        
        return unquote(title)
        #title = unquote(title)    
        
    def download_one_page(self,page_cnt,first_page,save_dir,page_now_ele):
        #check if page is actually the same with page_cnt
        
        while True:
            time.sleep(0.3)
            
            page_num_ele = driver.find_element_by_xpath('/html/body/div[2]/div/div[6]/div/div[2]/div/div[1]/div/div[2]/div')
            page_num_str = ""
            
            for i in page_num_ele.text:
                if i == '/':
                    break
                    
                page_num_str+= i
            
            page_num = int(page_num_str)
            
            if page_cnt == page_num:
                break
        
        
        loading = self.driver.find_elements_by_xpath('//div[contains(@class,"loading") and contains(@style,"visible")]')
        print("\n" + page_now_ele.text + " before :" + str(len(loading)))

        time.sleep(0.3)

        save_filename = str(page_cnt) + ".png"
        now_page = page_cnt % 2

        try:
            #wait for currentscreen loaded
            if page_cnt % 2 == first_page:
                viewport1_path = self.driver.find_element_by_xpath('//div[contains(@id,"viewport1") and contains(@class,"currentScreen")]')
                print(viewport1_path.get_attribute("id"))
                print(viewport1_path.get_attribute("class"))

                viewport1_wait = WebDriverWait(self.driver,15).until(
                    EC.presence_of_element_located((By.XPATH,'//div[contains(@id,"viewport1") and contains(@class,"currentScreen")]'))
                )
            else :
                viewport0_path = self.driver.find_element_by_xpath('//div[contains(@id,"viewport0") and contains(@class,"currentScreen")]')
                print(viewport0_path.get_attribute("id"))
                print(viewport0_path.get_attribute("class"))

                viewport0_wait = WebDriverWait(self.driver,15).until(
                    EC.presence_of_element_located((By.XPATH,'//div[contains(@id,"viewport0") and contains(@class,"currentScreen")]'))
                )

            time.sleep(0.3)
            
            #wait for all possible loading disappear
            
            for k in range(2):
            
                time.sleep(0.3)

                loading_wait1 = WebDriverWait(driver,20).until_not(
                    EC.visibility_of_element_located((By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div[2]'))
                )
                loading_wait2 = WebDriverWait(driver,20).until_not(
                    EC.visibility_of_element_located((By.XPATH,'/html/body/div[2]/div/div[2]/div[2]/div[3]'))
                )
                loading_wait3 = WebDriverWait(driver,20).until_not(
                    EC.visibility_of_element_located((By.XPATH,'/html/body/div[2]/div/div[2]/div[1]/div[2]'))
                )
                loading_wait4 = WebDriverWait(driver,20).until_not(
                    EC.visibility_of_element_located((By.XPATH,'/html/body/div[2]/div/div[2]/div[1]/div[3]'))
                )
            
                loading = driver.find_elements_by_xpath('//div[contains(@class,"loading") and contains(@style,"visible")]')
                
                if len(loading) == 0:
                    break
            
            loading = driver.find_elements_by_xpath('//div[contains(@class,"loading") and contains(@style,"visible")]')
            print(page_num_ele.text + " after :" + str(len(loading)))
            driver.save_screenshot(save_dir + "\\" + save_filename)
            #time.sleep(0.3)
            
            if len(loading) == 0 :
                ActionChains(driver).send_keys(Keys.LEFT).perform()
            else :
                page_cnt -= 1
            
        except :
            print("in except : " + str(page_cnt))
            return -1
        finally:
            print("Page : " + str(page_cnt))
            return 1
        
    def start_download_comic(self,comic_id):
        
        th = threading.Thread(target = self.download_comic,args = (comic_id,))
        th.setDaemon(True)
        th.start()
        self.window.after(20,self.check_thread_alive,th)
    
    def download_comic(self,comic_id):
        
        download_progress_window = tk.Toplevel(self.window,bg = self.bg)
        
        download_text_var = tk.StringVar()
        download_text_var.set("Your comic is downloading...")
        download_text = tk.Label(download_progress_window,textvariable = download_text_var,bg = self.bg,fg = self.fg,font = self.ft)
        download_text.pack(expand = True,fill = 'x')
        
        download_progress_bar = ttk.Progressbar(download_progress_window,orient='horizontal',length = 100,mode = 'determinate')
        download_progress_bar.pack()
        
        download_progress_bar['value'] = 0
        self.window.update_idletasks()
        
        self.driver.get(self.readable_link_list[comic_id])
        
        #wait for comic page loaded
        
        self.comic_page_loaded()
        
        #change showing two page to one
        
        self.two_page_to_one()

        #slide to the first page

        page_now_ele , total_page = self.to_comic_first_page()
        
        piece = 100 / total_page
        print("piece :",piece,"\n")
        
        #wait for page bar disappear

        self.comic_bar_disappear()

        #try to screenshot    

        title = self.new_dir_name(self.readable_link_list[comic_id])
        
        comic_code = 0
        
        if comic_id >= 0 and comic_id <= 9:
            comic_code = 0
        elif comic_id <= 12:
            comic_code = 1
        elif comic_id <= 15:
            comic_code = 2
        elif comic_id <= 18:
            comic_code = 3
        else :
            comic_code = 4

        save_dir ,page_cnt = self.make_dir(title,comic_code,total_page,download_text_var) #manga title,which category

        print("Download start at " + save_dir)
        print(title + " is downloading...")

        #get first page num
        first_page = self.first_page_class()
        print("first_page : " + str(first_page))

        #set_trace()
        page_cnt = 1

        while page_cnt <= total_page:
            #cls
            #clear()
            
            page_cnt += self.download_one_page(page_cnt,first_page,save_dir,page_now_ele)
            
            if page_cnt != total_page:
                download_progress_bar['value'] = page_cnt * piece
                print(download_progress_bar['value'])
                self.window.update_idletasks()
            else :
                download_progress_bar['value'] = 100
                self.window.update_idletasks()
        
        #clear()
        
        print("Download successfully")
        
        download_progress_window.destroy()
        
        #think twice about the return value
        return True
    
    #list comic part
    
    def add_login_cookie(self):
        fc = open('comicFuz.json')
        cookie = fc.read()
        cookie = json.loads(cookie)

        for c in cookie:
            if 'expiry' in c:
                del c['expiry']
            self.driver.add_cookie(c)

        self.driver.refresh()
        
    def start_list_of_readable_manga(self):
        
        loading_info = tk.Label(self.list_frame,text = "Next page is loading...",bg = self.bg,fg = self.fg,font = self.ft,height = 2)
        loading_info.place(relx = .47,rely = .4,anchor = 'c')
        #progressbar
        progress_bar = ttk.Progressbar(self.list_frame,orient='horizontal',length = 100,mode = 'determinate')
        progress_bar.place(relx = .47,rely = .5,anchor = 'c')
        
        progress_bar['value'] = 0
        self.window.update_idletasks()
        
        th = threading.Thread(target = self.list_of_readable_manga,args = (progress_bar,loading_info,))
        th.setDaemon(True)
        th.start()
        self.window.after(20,self.check_thread_alive,th)
    
    def list_of_readable_manga(self,progress_bar,loading_info):
        
        piece = 100 / 22
        loading = 0
        
        url = 'https://comic-fuz.com/series/'
        
        self.driver.get(url)
        
        self.add_login_cookie()
        
        #number(id) of cover
        cnt = 1
        readable_cover_link = []

        for series_num in range(1,6):

            if url[-1] == '/':
                url += str(series_num)
            else :
                url = url[:len(url) - 1:]
                url += str(series_num)

            print(url)
            self.driver.get(url)

            time.sleep(2)

            xpath = '//*[contains(@class,"list")]'
            comic_table = self.driver.find_element_by_xpath(xpath)
            #print(comic_table.get_attribute("class"))
            comic_list = comic_table.find_elements_by_tag_name("li")

            series_cnt = 0

            for item in comic_list:
                #check which li is it
                series_cnt += 1
                #check if it is rented
                if '読む' in item.text:
                    print(cnt," : ")
                    cnt += 1
                    
                    if cnt != 23:
                        progress_bar['value'] = loading + piece
                        loading += piece
                        self.window.update_idletasks()
                    else :
                        progress_bar['value'] = 100
                        self.window.update_idletasks()

                    title = item.find_element_by_class_name("title")
                    self.readable_manga_title.append(title.text)
                    print(title.text)

                    link_path = '/html/body/main/div/div/div/section/div/ul/li[' + str(series_cnt) + ']/div[2]/div[1]/div/a'
                    link = item.find_element_by_xpath(link_path)
                    self.readable_link_list.append(link.get_attribute("href"))

                    rimg = item.find_element_by_tag_name("img").get_attribute("src")
                    readable_cover_link.append(rimg)
                    display(iImage(url = rimg,width = 150,height = 215))
            
            print('finish')
        
        time.sleep(2)
        
        progress_bar.destroy()
        loading_info.destroy()
        #self.loading_frame.destroy()
        
        self.cover_link_to_image(readable_cover_link)
        self.list_all_in_frame()

        
window = tk.Tk()
window.title('ComicFuz')
window.geometry('500x300')
window.resizable(0,0)

chrome_options = Options()
#chrome_options.add_argument('headless')
chrome_options.add_argument('hide-scrollbars')
driver = webdriver.Chrome(options = chrome_options)

#setting new icon
icon = ImageTk.PhotoImage(file = 'Cover.png')
window.tk.call('wm','iconphoto',window._w,icon)
# print(icon)

comicfuz = ComicFuz(window,driver)

window.mainloop()

### problem
# [1.] can't download more than once

<class 'PIL.ImageTk.PhotoImage'>
112 112
noooooo
in login page
Login successfully!
https://comic-fuz.com/series/1
1  : 
週刊漫画ＴＩＭＥＳ　２０２０年２／２８号


2  : 
週刊漫画ＴＩＭＥＳ　２０２０年２／２１号


3  : 
週刊漫画ＴＩＭＥＳ　２０２０年２／１４号


4  : 
週刊漫画ＴＩＭＥＳ　２０２０年２／７号


5  : 
週刊漫画ＴＩＭＥＳ　２０２０年１／３１号


...

22  : 
まんがタイムきららフォワード　２０２０年１月号


finish
0 th cover complete
1 th cover complete
2 th cover complete
3 th cover complete
4 th cover complete
5 th cover complete
6 th cover complete
7 th cover complete
8 th cover complete
9 th cover complete
10 th cover complete
11 th cover complete
12 th cover complete
13 th cover complete
14 th cover complete
15 th cover complete
16 th cover complete
17 th cover complete
18 th cover complete
19 th cover complete
20 th cover complete
21 th cover complete
all complete
over
Waiting for page loaded.

Web is loaded

Two page to one page

page_now_ele.text : 2/207
page_now_str 2 <class 'str'>
Already to the first page.

piece : 0.4830917874396135 

Waiting for bar to disappear.

Already disappear.
Window size : 639 557 

create a folder D:\Picture\ComicFuz\まんがタイムきららキャラット\まんがタイムきららキャラット　２０２０年３月号 successfully

Download start at D:\Picture\ComicFuz\まんがタイムきららキャラット\まんがタイムきららキャラット　２０２０年３月号
まんがタイムきららキャラット　２０２０年３月号 is downloading...
first_page : 1

1/207 before :0
viewport1
currentScreen
1/207 afte

viewport1
currentScreen
85/207 after :0
Page : 85
41.54589371980676

86/207 before :0
viewport0
currentScreen
86/207 after :0
Page : 86
42.028985507246375

87/207 before :0
viewport1
currentScreen
87/207 after :0
Page : 87
42.51207729468599

88/207 before :0
viewport0
currentScreen
88/207 after :0
Page : 88
42.9951690821256

89/207 before :0
viewport1
currentScreen
89/207 after :0
Page : 89
43.47826086956522

90/207 before :0
viewport0
currentScreen
90/207 after :0
Page : 90
43.96135265700483

91/207 before :0
viewport1
currentScreen
91/207 after :0
Page : 91
44.44444444444444

92/207 before :0
viewport0
currentScreen
92/207 after :0
Page : 92
44.927536231884055

93/207 before :0
viewport1
currentScreen
93/207 after :0
Page : 93
45.41062801932367

94/207 before :0
viewport0
currentScreen
94/207 after :0
Page : 94
45.893719806763286

95/207 before :0
viewport1
currentScreen
95/207 after :0
Page : 95
46.3768115942029

96/207 before :0
viewport0
currentScreen
96/207 after :0
Page : 96
46.


178/207 before :0
viewport0
currentScreen
178/207 after :0
Page : 178
86.47342995169082

179/207 before :0
viewport1
currentScreen
179/207 after :0
Page : 179
86.95652173913044

180/207 before :0
viewport0
currentScreen
180/207 after :0
Page : 180
87.43961352657004

181/207 before :0
viewport1
currentScreen
181/207 after :0
Page : 181
87.92270531400966

182/207 before :0
viewport0
currentScreen
182/207 after :0
Page : 182
88.40579710144927

183/207 before :0
viewport1
currentScreen
183/207 after :0
Page : 183
88.88888888888889

184/207 before :0
viewport0
currentScreen
184/207 after :0
Page : 184
89.3719806763285

185/207 before :0
viewport1
currentScreen
185/207 after :0
Page : 185
89.85507246376811

186/207 before :0
viewport0
currentScreen
186/207 after :0
Page : 186
90.33816425120773

187/207 before :0
viewport1
currentScreen
187/207 after :0
Page : 187
90.82125603864733

188/207 before :0
viewport0
currentScreen
188/207 after :0
Page : 188
91.30434782608695

189/207 before :0
vie

Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\users\leo89\appdata\local\programs\python\python37-32\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "c:\users\leo89\appdata\local\programs\python\python37-32\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-6db7715d99f3>", line 792, in download_comic
    self.two_page_to_one()
  File "<ipython-input-1-6db7715d99f3>", line 584, in two_page_to_one
    one_page_hover.click().perform()
  File "c:\users\leo89\appdata\local\programs\python\python37-32\lib\site-packages\selenium\webdriver\common\action_chains.py", line 80, in perform
    self.w3c_actions.perform()
  File "c:\users\leo89\appdata\local\programs\python\python37-32\lib\site-packages\selenium\webdriver\common\actions\action_builder.py", line 76, in perform
    self.driver.execute(Command.W3C_ACTIONS, enc)
  File "c:\users\leo89\appdata\local\programs\python\python37-32\lib\s